2023년 4월 13일 동작 확인

# 1. KoGPT-2 Fine-Tuning 하기

In [ ]:
!pip install transformers

In [ ]:
import transformers
transformers.__version__

'4.35.2'

In [ ]:
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#사전 학습된 모델 불러오기
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', pad_token='<pad>')
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.7.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'lm_head.weight', 'transformer.h.1.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.0.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

In [ ]:
import pandas as pd
import tqdm
import urllib.request

In [ ]:
#train할 datset 불러오기
train_data = pd.read_csv('/content/drive/MyDrive/MBTI_PRJ/Dataset/gpt-2_data_generation/[4]P_J_dataset[total].csv')

In [ ]:
len(train_data)

592

In [ ]:
batch_size = 32

In [ ]:
#train dataset에 대한 Q와 A데이터로 분리 + 패딩
def get_chat_data():
  for question, answer in zip(train_data.Q.to_list(), train_data.A.to_list()):
    bos_token = [tokenizer.bos_token_id]
    eos_token = [tokenizer.eos_token_id]
    sent = tokenizer.encode('<usr>' + question + '<sys>' + answer)
    yield bos_token + sent + eos_token

In [ ]:
dataset = tf.data.Dataset.from_generator(get_chat_data, output_types=tf.int32)

In [ ]:
#패딩
dataset = dataset.padded_batch(batch_size=batch_size, padded_shapes=(None,), padding_values=tokenizer.pad_token_id)

In [ ]:
adam = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)

In [ ]:
steps = len(train_data) // batch_size + 1
print(steps)

19


In [ ]:
#준비한 데이터를 이용해 사전학습된 gpt-2 추가 학습하기(fine-tuning)
EPOCHS = 10

for epoch in range(EPOCHS):
  epoch_loss = 0

  for batch in tqdm.tqdm_notebook(dataset, total=steps):
      with tf.GradientTape() as tape:
          result = model(batch, labels=batch)
          loss = result[0]
          batch_loss = tf.reduce_mean(loss)

      grads = tape.gradient(batch_loss, model.trainable_variables)
      adam.apply_gradients(zip(grads, model.trainable_variables))
      epoch_loss += batch_loss / steps

  print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, epoch_loss))

<ipython-input-80-b8a0018a371a>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch in tqdm.tqdm_notebook(dataset, total=steps):


  0%|          | 0/19 [00:00<?, ?it/s]

[Epoch:    1] cost = 3.36517072


  0%|          | 0/19 [00:00<?, ?it/s]

[Epoch:    2] cost = 1.47894704


...

[Epoch:    8] cost = 0.548158705


  0%|          | 0/19 [00:00<?, ?it/s]

[Epoch:    9] cost = 0.499156892


  0%|          | 0/19 [00:00<?, ?it/s]

[Epoch:   10] cost = 0.487848371


In [ ]:
def return_answer_by_chatbot(user_text):
  sent = '<usr>' + user_text + '<sys>'
  input_ids = [tokenizer.bos_token_id] + tokenizer.encode(sent)
  input_ids = tf.convert_to_tensor([input_ids])
  output = model.generate(input_ids, max_length=50, do_sample=True, top_k=20)
  sentence = tokenizer.decode(output[0].numpy().tolist())
  chatbot_response = sentence.split('<sys> ')[1].replace('</s>', '')
  return chatbot_response

In [ ]:
#한 문장 확인하기
return_answer_by_chatbot('감정에 잘 휘둘리는 편인가요?')

'나는 감정보다는 즉흥적인 결정과 결정을 더 중요하게 생각해'

In [ ]:
#한 문장 확인하기
return_answer_by_chatbot('친구가 차 사고가 나서 연락이 왔다 나의 대답은?')

'나는 차사고 난 직후 계획에 없던 일이 생기면 불편함을 느끼나요?'

# 2. 답변 생성기

질문을 입력해주면, fine-tuning된 모델을 바탕으로 원하는 응답이 생성됩니다.</br></br>
단, 학습한 데이터의 양이 적을수록 GPT-2모델의 한계가 극명하게 드러납니다.</br>
(질문에 적합하지않은 답변이 꽤 생성되므로, 직접 해당 데이터를 삭제하는 절차를 반드시 거쳐야합니다.)

In [ ]:
#응답받고싶은 질문 입력하기
question_1 = '어떤 학문에 좁고 깊게 탐구하는 것을 좋아하는가 넓고 얕게 탐구하는 것을 좋아하는가?'
question_2 = '인생에서 가장 중요한 것은 무엇이라고 생각하나요?'

In [ ]:
####################################################################################################

In [ ]:
Answer = []

In [ ]:
#question_1에 대한 Answer Data 출력
for i in range(60):
  answer=return_answer_by_chatbot(question_1)

  if answer not in question_1:
    Answer.append(answer)

In [ ]:
#Answer 데이터를 엑셀로 저장하기
import pandas as pd

df = pd.DataFrame({'Answer':Answer})
df.to_excel('/content/drive/MyDrive/MBTI_PRJ/Dataset/Answer_data.xlsx')

In [ ]:
#답변개수 확인
len(Answer)

60

In [ ]:
####################################################################################################

In [ ]:
Answer_2 = []

In [ ]:
#question_2 에 대한 Answer Data 출력
for i in range(60):
  answer=return_answer_by_chatbot(question_2)

  if answer not in question_2:
    Answer_2.append(answer)

In [ ]:
#Answer 데이터를 엑셀로 저장하기
import pandas as pd

df = pd.DataFrame({'Answer':Answer_2})
df.to_excel('/content/drive/MyDrive/MBTI_PRJ/Dataset/Answer_data_2.xlsx')

In [ ]:
#생성된 답변 확인하기
Answer

['나는 사람들과의 대화보다는 각자의 생각이나 감정을 중요하게 생각한다',
 '사람들과의 대화보다는 각자의 관심사에 좀더 깊은 관심을 갖게 되는 것 같다',
 '나는 사람들과의 대화에 적극적으로 관심을 보인다',
 '나는 사람들 사이의 대화보다는 조용히 다가가는 편이다',
 '사람들과의 대화보다는 각자의 관심사에 더욱 깊은 관심을 가지게 돼요',
 '나는 보통 사람들과의 대화보다는 각자 자기계발에 더 관심이 많아요',
 '사람들과의 대화에서 자신의 얘기는 항상 숨기고 싶어요',
 '나는 사람들과의 대화를 통해 자신의 존재감을 느끼는 것이 편안해지는 기분이라 생각한다',
 '사람들과의 대화에서 자신의 이야기를 많이 나눈다',
 '사람들과 함께할 때 나의 세상이 더욱 활기차고 밝아지는 느낌이 든다',
 '나는 누구 앞에서 먼저 나서는 것보다는 주변의 반응을 먼저 살피는 편이야',
 '다른 사람들과의 대화와 교류보다는 나만의 생각에 잠기는 편이다',
 '사람들의 반응은 어떠한가요?',
 '사람들은 내가 사람들과의 대화에서 자주 웃음소리나 박수를 만들어내니 나만의 세계에 조금은 웃음기가 남다 필요한 것 같다',
 '사람들과의 대화에서도 나만의 생각과 감정을 깊게 공유한다',
 '나는 사람들과의 대화보다는 내 안의 생각의 깊이와 대화하는 시간이 늘 더욱 필요하다고 느껴지고 상대방의 이야기에 더욱 집중하는 것을 더 선호한다',
 '처음엔 낯뜨거울 것 같아요 ',
 '사람들과의 대화보다는 각자의 생각과 감정을 먼저 찾는 편이다',
 '다양한 사람들과의 대화를 통해 새로운 관점으로 생각을 하는 것이 중요해요',
 '나는 사람들 사이에 있을 때 내 세상이 더욱 활기차고 밝아지는 느낌을 받나요?',
 '보통 다른 사람들의 이야기를 듣는 것을 좋아해요',
 '사람들과의 대화와 교류보다 나만의 생각에 잠기게 되는 시간이 더욱 즐거워요',
 '사람들과의 대화나 행동은 항상 흥미로워요',
 '당신은 친구랑 만나기로 했는데 갑자기 약속이 취소되었을 때 기분이 어떠나요?',
 '많은 사람

In [ ]:
#한 개 답변 출력해보기
#return_answer_by_chatbot('질문 입력하기')
return_answer_by_chatbot('너는 누구야?')

# + 답변 대량생산을 위한 코드

In [ ]:
question_1 = '정리를 잘하는 편인가요?'
question_2 = '휴식을 취하기 전 할 일을 먼저 하고 쉬는 편인가요?'
question_3 = '일이 마음대로 안되면 어떤 생각이 드나요?'
question_4 = '일정이나 계획이 변경되는 것에 대해 스트레스를 받나요?'
question_5 = '일정표를 잘 작성하는 편인가요?'

In [ ]:
##############################################################################################################################

In [ ]:
Answer_1 = []
for i in range(60):
  answer=return_answer_by_chatbot(question_1)

  if answer not in question_1:
    Answer_1.append(answer)

In [ ]:
Answer_2 = []
for i in range(60):
  answer=return_answer_by_chatbot(question_2)

  if answer not in question_2:
    Answer_2.append(answer)

In [ ]:
Answer_3 = []
for i in range(60):
  answer=return_answer_by_chatbot(question_3)

  if answer not in question_3:
    Answer_3.append(answer)

In [ ]:
Answer_4 = []
for i in range(60):
  answer=return_answer_by_chatbot(question_4)

  if answer not in question_4:
    Answer_4.append(answer)

In [ ]:
Answer_5 = []
for i in range(60):
  answer=return_answer_by_chatbot(question_5)

  if answer not in question_5:
    Answer_5.append(answer)

In [ ]:
import pandas as pd

df = pd.DataFrame({'Answer':Answer_1})
df.to_excel('/content/drive/MyDrive/MBTI_PRJ/Dataset/gpt-2_data_generation/generated_data/Answer_1_data.xlsx')#ANswer 데이터를 엑셀로 저장하기

In [ ]:
df = pd.DataFrame({'Answer':Answer_2})
df.to_excel('/content/drive/MyDrive/MBTI_PRJ/Dataset/gpt-2_data_generation/generated_data/Answer_2_data.xlsx')#ANswer 데이터를 엑셀로 저장하기

In [ ]:
df = pd.DataFrame({'Answer':Answer_3})
df.to_excel('/content/drive/MyDrive/MBTI_PRJ/Dataset/gpt-2_data_generation/generated_data/Answer_3_data.xlsx')#ANswer 데이터를 엑셀로 저장하기

In [ ]:
df = pd.DataFrame({'Answer':Answer_4})
df.to_excel('/content/drive/MyDrive/MBTI_PRJ/Dataset/gpt-2_data_generation/generated_data/Answer_4_data.xlsx')#ANswer 데이터를 엑셀로 저장하기

In [ ]:
df = pd.DataFrame({'Answer':Answer_5})
df.to_excel('/content/drive/MyDrive/MBTI_PRJ/Dataset/gpt-2_data_generation/generated_data/Answer_5_data.xlsx')#ANswer 데이터를 엑셀로 저장하기